In [5]:
import numpy as np
import os
import cv2
import tensorflow as tf
import tensorflow.keras as K
import tensorflow.keras.layers as layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

In [110]:
TRAIN_DIR = './train'
VAL_DIR = './val'
EPOCHS = 10

In [10]:
trainImageGen = ImageDataGenerator(rescale = 1./255.)
valImageGen = ImageDataGenerator(rescale = 1./255.)

In [77]:
trainingSet = trainImageGen.flow_from_directory(batch_size = 64,
                                                directory = TRAIN_DIR,
                                                shuffle = True,
                                                target_size = (224, 224),
                                                class_mode = 'binary'
                                               )

valSet = valImageGen.flow_from_directory(batch_size = 64,
                                         directory = VAL_DIR,
                                         shuffle = False,
                                         target_size = (224, 224),
                                         class_mode = 'binary'
                                        )

Found 1376 images belonging to 2 classes.
Found 142 images belonging to 2 classes.


In [134]:
print(trainingSet.class_indices)

{'with_mask': 0, 'without_mask': 1}


In [62]:
# just seeing the mobilenet v2 architecture
tempModel = MobileNetV2()
tempModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [151]:
def getMaskModel():
    # take the mobilenet architecture and remove the last dense layer
    # this is called fine-tuning
    mobNet = MobileNetV2(weights = 'imagenet',
                         include_top = False,
                         input_tensor = (layers.Input(shape = (224, 224, 3)))
                        )
    # the following model made is after looking at the mobilenet v2 architecture and deciding which
    # layers to implement based on that
    model = mobNet.output   # taking the output spit out by the last layer - 1 
    # the last layer in our case is the average pooling layer
    # POOLING -> CONV -> BN -> RELU -> FLATTEN -> DENSE -> DROPOUT -> SOFTMAX
    model = layers.AveragePooling2D(pool_size = (7, 7))(model)
    
    model = layers.Flatten()(model)
    
    model = layers.Dense(units = 512, activation = 'relu')(model)
    model = layers.Dropout(rate = 0.5)(model)
    
    model = layers.Dense(units = 2, activation = 'softmax')(model)
    
    maskModel = K.models.Model(inputs = mobNet.input, outputs = model)
    
    for layer in mobNet.layers:
        layer.trainable = False
    
    return maskModel

In [152]:
K.backend.clear_session()
model = getMaskModel()

In [153]:
opt = K.optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)
model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

In [154]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

In [155]:
history = model.fit(trainingSet,
                    steps_per_epoch =  1376 // 64,
                    epochs = EPOCHS,
                    validation_data = valSet,
                    validation_steps = 142 // 64
                   )

Epoch 1/10
21/21 [==============================] - 45s 2s/step - loss: 0.9490 - accuracy: 0.4756 - val_loss: 0.7266 - val_accuracy: 0.6250
Epoch 2/10
21/21 [==============================] - 43s 2s/step - loss: 0.7064 - accuracy: 0.5244 - val_loss: 0.6942 - val_accuracy: 0.4688
Epoch 3/10
21/21 [==============================] - 43s 2s/step - loss: 0.6947 - accuracy: 0.4748 - val_loss: 0.6936 - val_accuracy: 0.4609
Epoch 4/10
21/21 [==============================] - 48s 2s/step - loss: 0.6938 - accuracy: 0.4764 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 5/10
15/21 [====================>.........] - ETA: 11s - loss: 0.6937 - accuracy: 0.4935

KeyboardInterrupt: 